# Processing GTFS data to Service Access Points (SAPS)
 * This notebook contains code that performs various tasks related to GTFS (General Transit Feed Specification) data processing.
 * The code is designed to handle, manipulate, and analyze GTFS data to support transit planning and operations.
 * It includes functionalities such as data cleaning, transformation, and visualization to provide insights into transit schedules and routes.

In a step-by-step guide, we will explain what steps are taken to achieve our final output (SAP features) that we need in our future research

## Importing necessary libraries and locating raw GTFS data
First, we have to import the necessary libraries and locate the GTFS data.


In [5]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import json

# locating each file that is necessary for the GTFS data
file_calendar_dates = 'gtfs-nl/calendar_dates.txt'
file_trips = 'gtfs-nl/trips.txt'
file_routes = 'gtfs-nl/routes.txt'
file_agency = 'gtfs-nl/agency.txt'
file_stops = 'gtfs-nl/stops.txt'
file_stop_times = 'gtfs-nl/stop_times.txt'

## Cleaning the data
Now, we need to clean all the raw data. This involves preprocessing each dataset to remove irrelevant columns and filter the data based on specific criteria.



### Calendar dates
First, we take a look at the `calendar_dates.txt` file. This file contains the column `service_id` with the combined `date`. Each date has multiple service_id's. So we need to extract a list with all those id's

In [6]:
# reading the file
df_calendar_dates_raw = pd.read_csv(file_calendar_dates, sep=',')
df_calendar_dates_raw.head()

,service_id,date,exception_type
0,1,20241202,1
1,2,20241202,1
2,2,20241214,1
3,3,20241202,1
4,3,20241213,1


#### Preprocessing of calendar dates
We drop the `exception_type` column, because it is irrelevant. Also, we convert the `date` column to a datetime format.

In [7]:
# Dropping the 'exception_type' column as it is irrelevant
df_calendar_dates = df_calendar_dates_raw.drop(['exception_type'], axis=1)

# Converting the 'date' column to datetime format
df_calendar_dates['date'] = pd.to_datetime(df_calendar_dates['date'], format='%Y%m%d')

# Displaying the first few rows of the cleaned dataframe
df_calendar_dates.head()


,service_id,date
0,1,2024-12-02
1,2,2024-12-02
2,2,2024-12-14
3,3,2024-12-02
4,3,2024-12-13


Now, we need to extract all the service_id`s for the date we selected: 02-12-2024

In [8]:
# selecting a specific date
selected_date = '2024-12-02'

# getting the service_ids for the selected date
df_calendar_dates_20241202 = df_calendar_dates[df_calendar_dates['date'] == '2024-12-02']
service_ids = df_calendar_dates_20241202['service_id'].unique()

df_calendar_dates_20241202.head()

,service_id,date
0,1,2024-12-02
1,2,2024-12-02
3,3,2024-12-02
5,4,2024-12-02
7,5,2024-12-02


### Trips
Now, we need to open the `trips.txt` file. This file contains all trips for each route. We need the trips that are on the selected day, so where the `service_id` is in `service_ids`

In [9]:
# reading the file
df_trips_raw = pd.read_csv(file_trips, sep=',')
df_trips_raw.head()

,route_id,service_id,trip_id,realtime_trip_id,trip_headsign,trip_short_name,trip_long_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,17522,515,218116770,IFF:HSN:1038,Rotterdam Centraal,1038,Intercity direct,1,NaN,1295593.0,0,NaN
1,17522,786,223294031,IFF:HSN:1018,Rotterdam Centraal,1018,Intercity direct,1,NaN,1295593.0,0,NaN
2,17522,823,223294078,IFF:HSN:1022,Rotterdam Centraal,1022,Intercity direct,1,NaN,1295593.0,0,NaN
3,17522,823,223294126,IFF:HSN:1028,Rotterdam Centraal,1028,Intercity direct,1,NaN,1295593.0,0,NaN
4,17522,668,223294185,IFF:HSN:1034,Rotterdam Centraal,1034,Intercity direct,1,NaN,1295593.0,0,NaN


#### Preprocessing of trips

We will drop all irrelevant columns: `realtime_trip_id`, `trip_short_name`, `trip_long_name`, `direction_id`, `block_id`, `shape_id`, `wheelchair_accessible` and `bikes_allowed`.
After that, we are only interested in the trips on the selected day. So the `service_id` has to be in `service_ids`

In [10]:
# Dropping irrelevant columns
df_trips_cleaned = df_trips_raw.drop(columns=['realtime_trip_id', 'trip_short_name', 'trip_long_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed'])

# Filtering trips on the selected day
df_trips = df_trips_cleaned[df_trips_cleaned['service_id'].isin(service_ids)]
df_trips.head()

,route_id,service_id,trip_id,trip_headsign
166,17522,113,257591988,Amsterdam Centraal
167,17522,113,257592039,Amsterdam Centraal
168,17522,113,257592091,Amsterdam Centraal
169,17522,113,257592195,Amsterdam Centraal
170,17522,1,257592802,Amsterdam Centraal


### Routes
the file `routes.txt` contains the transit routes. A route is a group of trips that are displayed to riders as a single service.We need this for the `agency_id` and for the `route_short_name` and `route_long_name`

In [11]:
# loading the routes file
df_routes_raw = pd.read_csv(file_routes, sep=',')
df_routes_raw.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color,route_text_color,route_url
0,104288,ALLGO,322,Lijn 322 Parkwijk - Amstel,NaN,3,e00713,ffffff,NaN
1,104290,ALLGO,327,Lijn 327 Haven - Amstel,NaN,3,e00713,ffffff,NaN
2,104291,ALLGO,330,Lijn 330 Almere Buiten - A'dam Bijlmer Arena,NaN,3,e00713,ffffff,NaN
3,104292,ALLGO,N22,Lijn N22 Leidseplein - Poort - Almere Buiten,NaN,3,f59c00,000000,NaN
4,104293,ALLGO,N23,Lijn N23 A'dam CS - Filmwijk - Almere Centrum,NaN,3,e00713,ffffff,NaN


#### Preprocessing of routes

For this file, we drop the `route_desc`, `route_color`, `route_text_color` and `route_url`

In [12]:
# Dropping irrelevant columns
df_routes = df_routes_raw.drop(columns=['route_desc', 'route_color', 'route_text_color', 'route_url'])
df_routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,104288,ALLGO,322,Lijn 322 Parkwijk - Amstel,3
1,104290,ALLGO,327,Lijn 327 Haven - Amstel,3
2,104291,ALLGO,330,Lijn 330 Almere Buiten - A'dam Bijlmer Arena,3
3,104292,ALLGO,N22,Lijn N22 Leidseplein - Poort - Almere Buiten,3
4,104293,ALLGO,N23,Lijn N23 A'dam CS - Filmwijk - Almere Centrum,3


Now, we have to change the digit of `route_type` to the name of the transport vehicle:

0 - Tram, Streetcar, Light rail. Any light rail or street level system within a metropolitan area.

1 - Subway, Metro. Any underground rail system within a metropolitan area.

2 - Rail. Used for intercity or long-distance travel.

3 - Bus. Used for short- and long-distance bus routes.

4 - Ferry. Used for short- and long-distance boat service.

In [13]:
# Mapping route_type values to descriptive values
route_type_mapping = {
    0: 'Tram',
    1: 'Metro',
    2: 'Trein',
    3: 'Bus',
    4: 'Ferry'
}

# Replacing route_type values in df_routes
df_routes['route_type'] = df_routes['route_type'].replace(route_type_mapping)
df_routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_type
0,104288,ALLGO,322,Lijn 322 Parkwijk - Amstel,Bus
1,104290,ALLGO,327,Lijn 327 Haven - Amstel,Bus
2,104291,ALLGO,330,Lijn 330 Almere Buiten - A'dam Bijlmer Arena,Bus
3,104292,ALLGO,N22,Lijn N22 Leidseplein - Poort - Almere Buiten,Bus
4,104293,ALLGO,N23,Lijn N23 A'dam CS - Filmwijk - Almere Centrum,Bus


### Agency
The file `agency.txt` contains the full name and information about the agency of the route. We need this for the full name of the agency.

In [14]:
# loading the agency file
df_agency_raw = pd.read_csv(file_agency, sep=',')
df_agency_raw.head()

,agency_id,agency_name,agency_url,agency_timezone,agency_phone
0,ALLGO,allGo (Keolis),http://www.allgobus.nl,Europe/Amsterdam,003188-0331360
1,ARR,Arriva,https://www.arriva.nl,Europe/Amsterdam,0031800-0232545
2,BRAVO:ARR,Bravo (Arriva),https://www.bravo.info,Europe/Amsterdam,0031800-0232545
3,BRAVO:CXX,Bravo (Hermes),http://www.bravo.info,Europe/Amsterdam,0031800-0222277
4,BRENG,Breng,http://www.breng.nl,Europe/Amsterdam,003126-2142140


#### Preprocessing of agency
We only need the full name (`agency_name`) combined with the `agency_id`. So we drop all other columns

In [15]:
# Dropping irrelevant columns
df_agency = df_agency_raw[['agency_id', 'agency_name']]
df_agency.head()

,agency_id,agency_name
0,ALLGO,allGo (Keolis)
1,ARR,Arriva
2,BRAVO:ARR,Bravo (Arriva)
3,BRAVO:CXX,Bravo (Hermes)
4,BRENG,Breng


### Stops
The file `stops.txt` is a dataset of each stop in the Netherlands, with the coordinates of the stop included.

In [16]:
# loading the stops file
df_stops_raw = pd.read_csv(file_stops, sep=',')
df_stops_raw.head()

C:\Users\emiel\AppData\Local\Temp\ipykernel_21000\2938655292.py:2: DtypeWarning: Columns (6,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stops_raw = pd.read_csv(file_stops, sep=',')


,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,stop_timezone,wheelchair_boarding,platform_code,zone_id
0,2323991,NaN,Koln-Ehrenfeld,50.951582,6.917458,0,stoparea:177908,NaN,NaN,NaN,IFF:kolne
1,2324425,NaN,Sappemeer Oost,53.158998,6.795339,0,stoparea:18130,NaN,NaN,1,IFF:spm
2,2324426,NaN,Sappemeer Oost,53.158848,6.796734,0,stoparea:18130,NaN,NaN,2,IFF:spm
3,2324427,NaN,Sappemeer Oost,53.158834,6.796141,0,stoparea:18130,NaN,NaN,NaN,IFF:spm
4,2380333,NaN,Veenwouden,53.235293,5.989464,0,stoparea:377859,NaN,NaN,NaN,IFF:vwd


#### Preprocessing of stops
For this file, we will drop the columns `stop_code`, `location_type`, `parent_station`, `stop_timezone`, `wheelchair_boarding`, `platform_code` and `zone_id`.

Some values in `stop_id` are not an integer. We looked manually at these stop_id's and these are duplicates and need to be removed.

In [17]:
# Dropping irrelevant columns
df_stops = df_stops_raw.drop(columns=['stop_code', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding', 'platform_code', 'zone_id'])

# Removing duplicates and non-integer stop_ids
df_stops = df_stops[df_stops['stop_id'].apply(lambda x: x.isdigit())]
df_stops['stop_id'] = df_stops['stop_id'].astype(int)
df_stops = df_stops.drop_duplicates(subset=['stop_id'])

df_stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,2323991,Koln-Ehrenfeld,50.951582,6.917458
1,2324425,Sappemeer Oost,53.158998,6.795339
2,2324426,Sappemeer Oost,53.158848,6.796734
3,2324427,Sappemeer Oost,53.158834,6.796141
4,2380333,Veenwouden,53.235293,5.989464


### Stop times
the file `stop_times.txt` is used for the times that a vehicle arrives and departs from an individual stop for each trip. It will link every file to each other. We are only focused on the `departure_time` between 17:00 and 18:00.

In [18]:
# loading the stop_times file
df_stop_times_raw = pd.read_csv(file_stop_times, sep=',')
df_stop_times_raw.head()

C:\Users\emiel\AppData\Local\Temp\ipykernel_21000\2971770227.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stop_times_raw = pd.read_csv(file_stop_times, sep=',')


,trip_id,stop_sequence,stop_id,stop_headsign,arrival_time,departure_time,pickup_type,drop_off_type,timepoint,shape_dist_traveled,fare_units_traveled
0,176957772,1,2862831,NaN,17:18:00,17:18:00,0,1,1,1.0,0
1,176957772,2,2862836,NaN,17:21:00,17:21:00,0,0,1,2695.0,0
2,176957772,3,2861628,NaN,17:29:00,17:30:00,0,0,1,11879.0,0
3,176957772,4,2861181,NaN,17:35:00,17:35:00,0,0,1,19619.0,0
4,176957772,5,2861973,NaN,17:41:00,17:42:00,0,0,1,28177.0,0


#### Preprocessing of stop times
For the preprocessing, we can drop the columns `stop_sequence`, `stop_headsign`, `arrival_time`, `pickup_type`, `drop_off_type`, `timepoint`, `shape_dist_traveled` and `fare_units_traveled`.

After that, we have to filter for the `departure_time` between 17:00:00 and 18:00:00. First, we have to convert the format to a time format. There are some values in `departure_time` above 23:59:59. We have to remove these, since these are irrelevant and we can not convert the column to time format.


In [19]:
# Dropping irrelevant columns
df_stop_times_cleaned = df_stop_times_raw.drop(columns=['stop_sequence', 'stop_headsign', 'arrival_time', 'pickup_type', 'drop_off_type', 'timepoint', 'shape_dist_traveled', 'fare_units_traveled'])

# Removing invalid departure times
df_stop_times_cleaned = df_stop_times_cleaned[df_stop_times_cleaned['departure_time'] <= '23:59:59']

# Converting departure_time to time format
df_stop_times_cleaned['departure_time'] = pd.to_datetime(df_stop_times_cleaned['departure_time'], format='%H:%M:%S').dt.time

# Filtering for departure_time between 17:00:00 and 18:00:00
df_stop_times = df_stop_times_cleaned[(df_stop_times_cleaned['departure_time'] >= pd.to_datetime('17:00:00').time()) &
                                               (df_stop_times_cleaned['departure_time'] <= pd.to_datetime('18:00:00').time())]
df_stop_times['departure_time'] = df_stop_times['departure_time'].apply(lambda x: x.strftime('%H:%M:%S'))
df_stop_times.head()

C:\Users\emiel\AppData\Local\Temp\ipykernel_21000\2121083636.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stop_times['departure_time'] = df_stop_times['departure_time'].apply(lambda x: x.strftime('%H:%M:%S'))


,trip_id,stop_id,departure_time
0,176957772,2862831,17:18:00
1,176957772,2862836,17:21:00
2,176957772,2861628,17:30:00
3,176957772,2861181,17:35:00
4,176957772,2861973,17:42:00


## Combining all data
Now that we have all the cleaned data, we need to combine it into one dataframe. For all dataframes, we have a unique key which can be linked:
- **df_calendar_dates_20241202**: Contains cleaned calendar dates data.
    - **Key**: `service_id`

- **df_trips**: Contains filtered trips data.
    - **Keys**: `routes_id`, `service_id`, `trip_id`

- **df_routes**: Contains cleaned routes data.
    - **Key**: `route_id`, `agency_id`

- **df_agency**: Contains cleaned agency data.
    - **Key**: `agency_id`

- **df_stops**: Contains cleaned stops data.
    - **Key**: `stop_id`

- **df_stop_times**: Contains filtered stop times data.
    - **Keys**: `trip_id`, `stop_id`


In [20]:
# Merging the agency in routes
df_routes_merged = pd.merge(df_routes, df_agency, on='agency_id', how='left')
df_routes_merged = df_routes_merged.drop(columns=['agency_id'])

# Merging the trips with calendar dates on 'service_id' and 'date'
df_trips_merged = pd.merge(df_trips, df_calendar_dates_20241202, on=['service_id'], how='left')

# Merging the routes with trips
df_routes_trips_merged = pd.merge(df_trips_merged, df_routes_merged, on='route_id', how='left')

# Merging the routes_trips with stop_times
df_routes_trips_stop_times_merged = pd.merge(df_routes_trips_merged, df_stop_times, on='trip_id', how='inner')

# Merging the stops with the combined dataframe
df_merged = pd.merge(df_routes_trips_stop_times_merged, df_stops, on='stop_id', how='left')

# Dropping unnecessary columns
df_merged = df_merged.drop(columns=['route_id', 'service_id', 'trip_id'])
df_merged.head()

,trip_headsign,date,route_short_name,route_long_name,route_type,agency_name,stop_id,departure_time,stop_name,stop_lat,stop_lon
0,Amsterdam Centraal,2024-12-02,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Trein,NS,2860858,17:05:00,Amsterdam Centraal,52.379431,4.899765
1,Amsterdam Centraal,2024-12-02,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Trein,NS,2862709,17:22:00,Schiphol Airport,52.309126,4.762030
2,Amsterdam Centraal,2024-12-02,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Trein,NS,2860855,17:35:00,Amsterdam Centraal,52.379386,4.899700
3,Amsterdam Centraal,2024-12-02,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Trein,NS,2862617,17:19:00,Rotterdam Centraal,51.925170,4.468454
4,Amsterdam Centraal,2024-12-02,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Trein,NS,2862709,17:52:00,Schiphol Airport,52.309126,4.762030


Now, we need to add the frequency per hour for each line.

In [21]:
# Group by stop_id and route_short_name and aggregate departure times
grouped = df_merged.groupby(['stop_id', 'trip_headsign', 'route_long_name'])['departure_time'].apply(lambda x: sorted(list(x))).reset_index()

# Counting the number of departure times for each combination
grouped['frequency'] = grouped['departure_time'].apply(len)

grouped.head()

,stop_id,trip_headsign,route_long_name,departure_time,frequency
0,2470552,Den Haag via Naaldwijk,Den Haag Leyenburg via Naaldwijk - Schiedam,"[17:00:00, 17:12:00, 17:31:00, 17:49:00]",4
1,2470553,Schiedam via Naaldwijk,Den Haag Leyenburg via Naaldwijk - Schiedam,"[17:12:00, 17:28:00, 17:42:00, 17:57:00]",4
2,2470596,Maassluis via Naaldwijk,Monster via Naaldwijk - Maassluis,"[17:07:33, 17:36:53]",2
3,2470597,Monster via Naaldwijk,Monster via Naaldwijk - Maassluis,"[17:19:26, 17:52:25]",2
4,2470598,Maassluis via Naaldwijk,Monster via Naaldwijk - Maassluis,"[17:09:05, 17:38:21]",2


Now, we merge the df_merged and grouped together. Also we add the frequency column and drop the duplicate rows. Since a DF with a list in it can not be dropped, we convert it to a JSON string, drop the duplicates, and convert it back.

In [22]:
# Selecting the relevant columns
df_final = df_merged[['stop_id', 'route_short_name', 'route_long_name', 'trip_headsign', 'route_type', 'agency_name', 'stop_lat', 'stop_lon', 'date']]

# Adding the frequency column
df_final = pd.merge(df_final, grouped[['stop_id', 'trip_headsign', 'route_long_name', 'frequency', 'departure_time']], on=['stop_id', 'trip_headsign', 'route_long_name'], how='left')

df_final['departure_time'] = df_final['departure_time'].apply(json.dumps)
df_final = df_final.drop_duplicates()
df_final['departure_time'] = df_final['departure_time'].apply(json.loads)

df_final.head()

,stop_id,route_short_name,route_long_name,trip_headsign,route_type,agency_name,stop_lat,stop_lon,date,frequency,departure_time
0,2860858,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Amsterdam Centraal,Trein,NS,52.379431,4.899765,2024-12-02,1,[17:05:00]
1,2862709,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Amsterdam Centraal,Trein,NS,52.309126,4.762030,2024-12-02,2,"[17:22:00, 17:52:00]"
2,2860855,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Amsterdam Centraal,Trein,NS,52.379386,4.899700,2024-12-02,1,[17:35:00]
3,2862617,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Amsterdam Centraal,Trein,NS,51.925170,4.468454,2024-12-02,2,"[17:19:00, 17:50:00]"
6,2862617,Intercity direct,Rotterdam Centraal <-> Amsterdam Centraal HSN1000,Rotterdam Centraal,Trein,NS,51.925170,4.468454,2024-12-02,2,"[17:07:00, 17:40:00]"


## Saving the output

Now that we have the final dataframe, we want to save it as a CSV file.

In [23]:
# saving the final dataframe to a csv file
df_final.to_csv('OV_stops_time.csv', index=False)